In [7]:
import itertools as it
import pandas as pd
import requests as rq
from IPython.display import display

In [12]:
# Parameters
experiments=1
users = "../data/users.json"
host = "localhost"
port = "5000"
sample_size = 500
limits = [10, 50]
walks = [1000]
walk_length = [100]

In [13]:
# Import users
user_df = pd.read_json(users, orient="table")
display(user_df)

# Compute cross product of all configuration values
configurations = list(it.product(walks, walk_length, limits))

,user
2267238,4671229608
1396952,2312687048
5066612,228541032
3807989,946097265625538560
2680420,1018562002678108162
...,...
748630,1597388371
1156305,2696695362
5014030,1330692817
962078,2987955914


In [21]:
def get_sample(df, offset, sample_size):
    return df.iloc[offset:offset + sample_size]

def get_recommendations(sample_df, walks, walk_length, limit):
    rows = []

    for i, row in sample_df.iterrows():
        user = row["user"]
        url = f"http://{host}:{port}/recommendation/salsa/{user}?walks={walks}&walk_length={walk_length}&limit={limit}"
        response = rq.get(url)

        status_code = response.status_code
        duration = response.elapsed.total_seconds()
        recommendations = response.json() if response.status_code == 200 else []
        rows.append((walks, walk_length, limit, url, status_code, duration, user, recommendations))
    
    return pd.DataFrame(rows, columns=["walks", "walk_length", "limit", "url", "status_code", "duration", "user", "recommendations"])

In [22]:
# Perform API requests

for experiment in range(1):

    offset = 0
    result_dfs = []

    for (walks, walk_length, limit) in configurations:
        print(f"Fetch recommendations: walks {walks}, walk_length {walk_length}, limit {limit}")
#         sample_df = get_sample(user_df, offset, sample_size)
        result_dfs.append(get_recommendations(user_df, walks, walk_length, limit))
        offset = offset + sample_size

    result_df = pd.concat(result_dfs)
    display(result_df)

    # Save results
    output = f"../data/recommendations_{experiment}.json"
    result_df.to_json(output, index=False, orient="table")
    result_df.head(50)

Fetch recommendations: walks 1000, walk_length 100, limit 10
Fetch recommendations: walks 1000, walk_length 100, limit 50


,walks,walk_length,limit,url,status_code,duration,user,recommendations
0,1000,100,10,http://localhost:5000/recommendation/salsa/467...,200,0.560005,4671229608,"[1128375181368541184, 1134175400794480640, 113..."
1,1000,100,10,http://localhost:5000/recommendation/salsa/231...,200,0.859831,2312687048,"[1130508261076160513, 1133805499521146880, 113..."
2,1000,100,10,http://localhost:5000/recommendation/salsa/228...,200,0.514798,228541032,"[1133920658994409472, 1134302923226267648, 113..."
3,1000,100,10,http://localhost:5000/recommendation/salsa/946...,200,0.686450,946097265625538560,"[1133750110603628551, 1133025240039211010, 113..."
4,1000,100,10,http://localhost:5000/recommendation/salsa/101...,200,0.559290,1018562002678108162,"[1124998188513890306, 1134690913031057409, 113..."
...,...,...,...,...,...,...,...,...
495,1000,100,50,http://localhost:5000/recommendation/salsa/159...,200,0.703756,1597388371,"[1133828151774199808, 1133052518160896000, 113..."
496,1000,100,50,http://localhost:5000/recommendation/salsa/269...,200,0.829195,2696695362,"[1134097908092063745, 1132995158524477440, 104..."
497,1000,100,50,http://localhost:5000/recommendation/salsa/133...,200,0.528345,1330692817,"[1129431074210291712, 1133701058843238409, 112..."
498,1000,100,50,http://localhost:5000/recommendation/salsa/298...,200,0.512423,2987955914,"[1133992185710333952, 1127997544183631872, 113..."
